# Importando as bibliotecas

In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
from collections import Counter
import os
import sys
sys.path.insert(0, os.path.abspath('../Util'))
from dados import ProcessarDados
#!python -m pip install --user scikit-mlm

from skmlm import NN_MLM, MLMC

#from skmlm import FCM_MLM as NN_MLM
import time

# Dataset

In [35]:
procData = ProcessarDados("../dataset/bin_norm_10_features_m73_n4.csv", sep = ',')

# Funções

In [36]:
# funcção que treina e testa o modelo armazenando as métricas
# retorna um dicionário cotendo os valores das métricas de cada rodada
def treinamento_teste(epocas = 10, k_folds = 5, exibir_matriz_confusao=False, exibir_metricas=False):
    
    #array para armazenar as das métricas de cada rodada
    resultados_accuracy = []
    resultados_precision = []
    resultados_recall = []
    resultados_specificity = [] # taxa de verdadeiros negativos ou especificidade
    resultados_f2 = []
    resultados_parametros = []
    resultados_time_train = []
    resultados_time_test = []
    resultados_time_total = []
    
    #dicionário das métricas
    resultados_gerais = {}

    for i in range(epocas):
        # divisão os dados 
        seed = i
        X_train, X_test, y_train, y_test = procData.holdout2(0.2, seed)
        #print(Counter(y_test))

        # realizando o grid search para encontrar a melhor combinação de parametros, 
        # considerando a acurácia (taxa de acerto)
        # aqui o método GridSearchCV é configurado para subdividir os dados de treino em k_folds
        clf = NN_MLM(random_state=seed)
        grid_mlm = GridSearchCV(clf, param_grid, cv=k_folds, scoring='accuracy', verbose=0)
        
        start_time_train = time.clock()
        grid_mlm.fit(X_train, y_train)

        # Treinando do modelo com os melhores parametros encontrados
        rp_number_best = grid_mlm.best_estimator_.rp_number

        MLM = NN_MLM(rp_number = rp_number_best, random_state=seed)
        MLM.fit(X_train, y_train)

        time_train = time.clock() - start_time_train
        
        #testando o modelo
        start_time_test = time.clock()
        y_pred = MLM.predict(X_test)
        time_test = time.clock() - start_time_test
        cm  = confusion_matrix(y_test, y_pred)
        if exibir_matriz_confusao:
            print(cm)

        # calculado as metricas
        accuracy = metrics.accuracy_score(y_test, y_pred)
        precision = metrics.precision_score(y_test, y_pred)
        recall = metrics.recall_score(y_test, y_pred)
        # f2-score
        # Fbeta = ((1 + beta^2) * Precision * Recall) / (beta^2 * Precision + Recall)
        beta = 0.5
        f2_score = (1 + beta**2) * (precision * recall) / (beta**2 * precision + recall)
        tn, fp, fn, tp = cm.ravel()
        specificity = tn / (tn+fp)

        # armazenando as métricas
        resultados_accuracy.append(accuracy)
        resultados_precision.append(precision)
        resultados_recall.append(recall)
        resultados_specificity.append(specificity)
        resultados_f2.append(f2_score)

        best_parametros = {"rp_number": rp_number_best};
        resultados_parametros.append(best_parametros)
        
        resultados_time_train.append(time_train)
        resultados_time_test.append(time_test) 
        resultados_time_total.append(time_train+time_test)


        if exibir_metricas:
            print("Rodada: #",i)
            print(best_parametros)
            print("Accuracy:",accuracy)
            print("Precision:",precision)
            print("Recall:",recall)
            print("Specificity:",specificity)
            print("f2-Score:",f2_score)
            print("Time Train (s):",time_train)
            print("Time Test (s):",time_test)
            print("Time Total (s):",time_train+time_test)
            print("\n")

            
    resultados_gerais['accuracy'] = resultados_accuracy
    resultados_gerais['precision'] = resultados_precision
    resultados_gerais['recall'] = resultados_recall
    resultados_gerais['specificity'] = resultados_specificity
    resultados_gerais['f2'] = resultados_f2
    resultados_gerais['params'] = resultados_parametros
    resultados_gerais['time_train'] = resultados_time_train
    resultados_gerais['time_test'] = resultados_time_test
    resultados_gerais['time_total'] = resultados_time_total
    
    return resultados_gerais

In [37]:
import csv
def writeResultsCsv(dict_results, name_file):
    keys = list(dict_results.keys())
    keys.insert(0,'Rodada')
    with open(name_file, 'w') as f:  # You will need 'wb' mode in Python 2.x
        w = csv.writer(f)
        #w.writeheader()
        row = []
        w.writerow(keys)
        for i in range(epocas):
            row = []
            acc = dict_results.get('accuracy')[i]
            prec = dict_results.get('precision')[i]
            sen = dict_results.get('recall')[i]
            spe = dict_results.get('specificity')[i]
            fscore = dict_results.get('f2')[i]
            param = dict_results.get('params')[i]
            time_train = dict_results.get('time_train')[i]
            time_test = dict_results.get('time_test')[i]
            time_total = dict_results.get('time_total')[i]

            row.append(i+1)
            row.append(acc)
            row.append(prec)
            row.append(sen)
            row.append(spe)
            row.append(fscore)
            row.append(param)
            row.append(time_train)
            row.append(time_test)
            row.append(time_total)

            w.writerow(row)
    

In [38]:
def tabelaMetricas(nome_modelo, dict_metricas, rodadas=False, salvarResultados=True):
        
    print ("============================================== "+nome_modelo+" =================================================")
    print ("=================================== TABELA DE MÉTRICAS DO MODELO ===================================")
    
    if(rodadas==False):
        print ("\t Accuracy \t|\t Precision \t|\t Recall \t|\tSpecificity \t|\t fb-Score")
        print ("      %.4f +- %.4f" % (np.mean(dict_metricas['accuracy'], axis=0), np.std(dict_metricas['accuracy'], axis=0)),end=' ')
        print ("      %.4f +- %.4f" % (np.mean(dict_metricas['precision'], axis=0), np.std(dict_metricas['precision'], axis=0)),end='  ')
        print ("      %.4f +- %.4f" % (np.mean(dict_metricas['recall'], axis=0), np.std(dict_metricas['recall'], axis=0)),end=' ')
        print ("      %.4f +- %.4f" % (np.mean(dict_metricas['specificity'], axis=0), np.std(dict_metricas['specificity'], axis=0)),end='   ')
        print ("      %.4f +- %.4f" % (np.mean(dict_metricas['f2'], axis=0), np.std(dict_metricas['f2'], axis=0)))
        print ("====================================================================================================")
        
    if(salvarResultados):
        # save to npy file
        np.save('../resultados/resultados_'+nome_modelo+'.npy', dict_metricas)
        writeResultsCsv(dict_metricas, 'resultados_mlm.csv')

# Definindo os parâmetros

In [39]:
# pontos de referencia
rp_number = np.arange(0.7,0.8,0.1)
param_grid = {'rp_number': rp_number}

epocas = 50
k_folds = 5
exibir_matriz_confusao = True
exibir_metricas = True
salvarResultados = True
rodadas=False

# Treinando e obtendo as métricas do modelo

In [40]:
# treinando o modelo
dict_metricas = treinamento_teste(epocas, k_folds, exibir_matriz_confusao, exibir_metricas)
tabelaMetricas('MLM',dict_metricas, rodadas, salvarResultados)

[[18  2]
 [ 2 48]]
Rodada: # 0
{'rp_number': 223}
Accuracy: 0.9428571428571428
Precision: 0.96
Recall: 0.96
Specificity: 0.9
f2-Score: 0.96
Time Train (s): 0.20693589999973483
Time Test (s): 0.000930500000322354
Time Total (s): 0.20786640000005718


[[18  2]
 [ 3 47]]
Rodada: # 1
{'rp_number': 196}
Accuracy: 0.9285714285714286
Precision: 0.9591836734693877
Recall: 0.94
Specificity: 0.9
f2-Score: 0.9552845528455283
Time Train (s): 0.08802260000084061
Time Test (s): 0.0008163000002241461
Time Total (s): 0.08883890000106476


[[17  3]
 [ 1 49]]
Rodada: # 2
{'rp_number': 196}
Accuracy: 0.9428571428571428
Precision: 0.9423076923076923
Recall: 0.98
Specificity: 0.85
f2-Score: 0.9496124031007752
Time Train (s): 0.1505488000002515
Time Test (s): 0.0005952999999863096
Time Total (s): 0.1511441000002378


[[18  2]
 [ 3 47]]
Rodada: # 3
{'rp_number': 196}
Accuracy: 0.9285714285714286
Precision: 0.9591836734693877
Recall: 0.94
Specificity: 0.9
f2-Score: 0.9552845528455283
Time Train (s): 0.0957692